In [1]:
import pandas as pd
from utils.utils import preprocessing, date_column_change

In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', None)

In [3]:
data = pd.read_csv("Data/пп_станции.xlsx - Лист1.csv")
data.head()

,Станция,Номер линии,Дата,4/3/2024,4/2/2024,4/1/2024,3/31/2024,3/30/2024,3/29/2024,3/28/2024,3/27/2024,3/26/2024,3/25/2024,3/24/2024,3/23/2024,3/22/2024,3/21/2024,3/20/2024,3/19/2024,3/18/2024,3/17/2024,3/16/2024,3/15/2024,3/14/2024,3/13/2024,3/12/2024,3/11/2024,3/10/2024,3/9/2024,3/8/2024,3/7/2024,3/6/2024,3/5/2024,3/4/2024,3/3/2024,3/2/2024,3/1/2024,2/29/2024,2/28/2024,2/27/2024,2/26/2024,2/25/2024,2/24/2024,2/23/2024,2/22/2024,2/21/2024,2/20/2024,2/19/2024,2/18/2024,2/17/2024,2/16/2024,2/15/2024,2/14/2024,2/13/2024,2/12/2024,2/11/2024,2/10/2024,2/9/2024,2/8/2024,2/7/2024,2/6/2024,2/5/2024,2/4/2024,2/3/2024,2/2/2024,2/1/2024,1/31/2024,1/30/2024,1/29/2024,1/28/2024,1/27/2024,1/26/2024,1/25/2024,1/24/2024,1/23/2024,1/22/2024,1/21/2024,1/20/2024,1/19/2024,1/18/2024,1/17/2024,1/16/2024,1/15/2024,1/14/2024,1/13/2024,1/12/2024,1/11/2024,1/10/2024,1/9/2024,1/8/2024,1/7/2024,1/6/2024,1/5/2024,1/4/2024,1/3/2024,1/2/2024,1/1/2024
0,Б.Рокоссовского,8,СОКОЛЬНИЧЕСКАЯ,16671,15113,2905,13998,12742,1217,17624,12097,14831,7792,9609,1229,2135,6895,1041,18651,4943,5619,17125,19679,8482,3261,8065,17639,7450,2673,1088,5490,11985,5938,18694,6368,5575,17447,2239,2812,11670,5146,19224,10012,6214,8652,18346,13882,1888,18150,11240,12338,13991,18355,1248,4186,18087,14410,16187,9018,9734,4153,10858,3697,18317,17464,7277,4205,12204,3369,19395,1223,14351,2600,14185,7410,18924,10265,3061,1510,10798,19898,7645,17944,16665,14355,10499,18189,7280,4669,3812,5816,15740,13714,2561,11209,2722,4835
1,Черкизовская,8,СОКОЛЬНИЧЕСКАЯ,6248,5441,7991,14864,10015,9721,6623,13030,12372,17814,15379,2339,14576,18593,2588,6929,7277,3129,17656,1294,4426,4148,14293,16764,15338,6452,12030,8008,17510,16592,4520,18700,8816,2181,15752,14542,5213,4556,12789,5340,17570,3025,3865,8765,7515,17010,15248,3176,12324,15487,16393,12995,9768,16338,12248,17086,18639,14294,14153,12435,12251,2947,11779,14577,18868,17624,16820,11171,10935,2485,13333,11854,1667,5797,12029,1828,6468,12877,7645,15416,17947,9090,3849,17700,13486,5061,3663,11353,6924,6496,15991,7029,3999,15638
2,Преображенск. пл,8,СОКОЛЬНИЧЕСКАЯ,14094,1132,6556,7265,4001,10961,6852,13961,11940,2349,19378,14975,12584,3391,12367,7694,5643,14169,6267,7095,2167,7788,17924,9286,13476,14517,9040,12822,7241,8939,9775,10587,8814,19336,11912,8303,6590,1276,17036,11525,2127,3033,6349,4682,10830,16736,4526,19907,1634,6083,9522,15279,9890,19921,6404,6843,16351,12981,3417,11412,16221,17054,10516,15896,17579,14727,15152,8801,1530,7813,7735,10134,14017,8169,10657,12490,18196,5698,18122,10740,12682,9135,19574,4361,8176,9745,13281,15090,13309,8365,2811,5004,17068,17117
3,Сокольники СЛ,8,СОКОЛЬНИЧЕСКАЯ,18651,10083,6488,6409,9446,12051,15041,6793,3010,19196,17958,13304,19168,16050,8283,1348,8076,9769,13495,13672,16816,18654,7779,11842,4898,7845,8019,11014,7822,13541,19996,18196,13928,10224,2236,12909,9933,6320,5646,19848,17931,10014,11184,6686,16012,3537,10608,12939,19224,11326,18973,1272,6400,19425,1662,17534,19227,11321,1465,9050,17897,2104,4598,9385,2009,7977,7327,16742,9381,3279,15963,10741,6518,2690,16002,15438,18937,11864,19236,7969,12214,8669,7326,7890,3637,14001,12049,15972,15545,18759,5573,15017,14176,5465
4,Красносельская,8,СОКОЛЬНИЧЕСКАЯ,7391,19339,15424,10725,5965,14682,18134,2055,9929,9980,10764,5599,9113,4628,11462,18312,18938,3711,10294,13716,8803,1803,13435,2167,6980,3814,12569,11659,14564,15689,1267,13190,13247,18587,17385,19361,10486,5850,15293,11900,14056,17564,2791,19596,5429,4949,9723,14583,15674,7914,4116,5767,10763,17830,10085,8934,8231,17688,12619,8573,9204,11679,5864,3054,7617,12718,4241,16727,8937,13782,18008,14760,2836,1210,8202,17022,11939,7065,8814,7338,7490,3529,17303,8012,14194,7615,9321,16152,4581,7746,8936,11373,19561,1727


In [4]:
data = preprocessing(data)
data.head()

/Users/alexander/Учёба/ХАКАТОНЫ/Nuclear Hack/nuclear-hack/utils/utils.py:308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Станция'] = df['Станция'].replace(rename_dict)


,Станция,Номер линии,Линия,01-01-2024,02-01-2024,03-01-2024,04-01-2024,05-01-2024,06-01-2024,07-01-2024,08-01-2024,09-01-2024,10-01-2024,11-01-2024,12-01-2024,13-01-2024,14-01-2024,15-01-2024,16-01-2024,17-01-2024,18-01-2024,19-01-2024,20-01-2024,21-01-2024,22-01-2024,23-01-2024,24-01-2024,25-01-2024,26-01-2024,27-01-2024,28-01-2024,29-01-2024,30-01-2024,31-01-2024,01-02-2024,02-02-2024,03-02-2024,04-02-2024,05-02-2024,06-02-2024,07-02-2024,08-02-2024,09-02-2024,10-02-2024,11-02-2024,12-02-2024,13-02-2024,14-02-2024,15-02-2024,16-02-2024,17-02-2024,18-02-2024,19-02-2024,20-02-2024,21-02-2024,22-02-2024,23-02-2024,24-02-2024,25-02-2024,26-02-2024,27-02-2024,28-02-2024,29-02-2024,01-03-2024,02-03-2024,03-03-2024,04-03-2024,05-03-2024,06-03-2024,07-03-2024,08-03-2024,09-03-2024,10-03-2024,11-03-2024,12-03-2024,13-03-2024,14-03-2024,15-03-2024,16-03-2024,17-03-2024,18-03-2024,19-03-2024,20-03-2024,21-03-2024,22-03-2024,23-03-2024,24-03-2024,25-03-2024,26-03-2024,27-03-2024,28-03-2024,29-03-2024,30-03-2024,31-03-2024,01-04-2024,02-04-2024,03-04-2024
0,Бульвар Рокоссовского,8.0,СОКОЛЬНИЧЕСКАЯ,4835.0,2722.0,11209.0,2561.0,13714.0,15740.0,5816.0,3812.0,4669.0,7280.0,18189.0,10499.0,14355.0,16665.0,17944.0,7645.0,19898.0,10798.0,1510.0,3061.0,10265.0,18924.0,7410.0,14185.0,2600.0,14351.0,1223.0,19395.0,3369.0,12204.0,4205.0,7277.0,17464.0,18317.0,3697.0,10858.0,4153.0,9734.0,9018.0,16187.0,14410.0,18087.0,4186.0,1248.0,18355.0,13991.0,12338.0,11240.0,18150.0,1888.0,13882.0,18346.0,8652.0,6214.0,10012.0,19224.0,5146.0,11670.0,2812.0,2239.0,17447.0,5575.0,6368.0,18694.0,5938.0,11985.0,5490.0,1088.0,2673.0,7450.0,17639.0,8065.0,3261.0,8482.0,19679.0,17125.0,5619.0,4943.0,18651.0,1041.0,6895.0,2135.0,1229.0,9609.0,7792.0,14831.0,12097.0,17624.0,1217.0,12742.0,13998.0,2905.0,15113.0,16671.0
1,Черкизовская,8.0,СОКОЛЬНИЧЕСКАЯ,15638.0,3999.0,7029.0,15991.0,6496.0,6924.0,11353.0,3663.0,5061.0,13486.0,17700.0,3849.0,9090.0,17947.0,15416.0,7645.0,12877.0,6468.0,1828.0,12029.0,5797.0,1667.0,11854.0,13333.0,2485.0,10935.0,11171.0,16820.0,17624.0,18868.0,14577.0,11779.0,2947.0,12251.0,12435.0,14153.0,14294.0,18639.0,17086.0,12248.0,16338.0,9768.0,12995.0,16393.0,15487.0,12324.0,3176.0,15248.0,17010.0,7515.0,8765.0,3865.0,3025.0,17570.0,5340.0,12789.0,4556.0,5213.0,14542.0,15752.0,2181.0,8816.0,18700.0,4520.0,16592.0,17510.0,8008.0,12030.0,6452.0,15338.0,16764.0,14293.0,4148.0,4426.0,1294.0,17656.0,3129.0,7277.0,6929.0,2588.0,18593.0,14576.0,2339.0,15379.0,17814.0,12372.0,13030.0,6623.0,9721.0,10015.0,14864.0,7991.0,5441.0,6248.0
2,Преображенская площадь,8.0,СОКОЛЬНИЧЕСКАЯ,17117.0,17068.0,5004.0,2811.0,8365.0,13309.0,15090.0,13281.0,9745.0,8176.0,4361.0,19574.0,9135.0,12682.0,10740.0,18122.0,5698.0,18196.0,12490.0,10657.0,8169.0,14017.0,10134.0,7735.0,7813.0,1530.0,8801.0,15152.0,14727.0,17579.0,15896.0,10516.0,17054.0,16221.0,11412.0,3417.0,12981.0,16351.0,6843.0,6404.0,19921.0,9890.0,15279.0,9522.0,6083.0,1634.0,19907.0,4526.0,16736.0,10830.0,4682.0,6349.0,3033.0,2127.0,11525.0,17036.0,1276.0,6590.0,8303.0,11912.0,19336.0,8814.0,10587.0,9775.0,8939.0,7241.0,12822.0,9040.0,14517.0,13476.0,9286.0,17924.0,7788.0,2167.0,7095.0,6267.0,14169.0,5643.0,7694.0,12367.0,3391.0,12584.0,14975.0,19378.0,2349.0,11940.0,13961.0,6852.0,10961.0,4001.0,7265.0,6556.0,1132.0,14094.0
3,Сокольники,8.0,СОКОЛЬНИЧЕСКАЯ,5465.0,14176.0,15017.0,5573.0,18759.0,15545.0,15972.0,12049.0,14001.0,3637.0,7890.0,7326.0,8669.0,12214.0,7969.0,19236.0,11864.0,18937.0,15438.0,16002.0,2690.0,6518.0,10741.0,15963.0,3279.0,9381.0,16742.0,7327.0,7977.0,2009.0,9385.0,4598.0,2104.0,17897.0,9050.0,1465.0,11321.0,19227.0,17534.0,1662.0,19425.0,6400.0,1272.0,18973.0,11326.0,19224.0,12939.0,10608.0,3537.0,16012.0,6686.0,11184.0,10014.0,17931.0,19848.0,5646.0,6320.0,9933.0,12909.0,2236.0,10224.0,13928.0,18196.0,19996.0,13541.0,7822.0,11014.0,8019.0,7845.0,4898.0,11842.0,7779.0,18654.0,16816.0,13672.0,13495.0,9769.0,8076.0,1348.0,8283.0,16050.0,19168.0,13304.0,17958.0,19196.0,3010.0,6793.0,15041.0,12

In [5]:
data[data["Станция"] == "Нижегородская"]

,Станция,Номер линии,Линия,01-01-2024,02-01-2024,03-01-2024,04-01-2024,05-01-2024,06-01-2024,07-01-2024,08-01-2024,09-01-2024,10-01-2024,11-01-2024,12-01-2024,13-01-2024,14-01-2024,15-01-2024,16-01-2024,17-01-2024,18-01-2024,19-01-2024,20-01-2024,21-01-2024,22-01-2024,23-01-2024,24-01-2024,25-01-2024,26-01-2024,27-01-2024,28-01-2024,29-01-2024,30-01-2024,31-01-2024,01-02-2024,02-02-2024,03-02-2024,04-02-2024,05-02-2024,06-02-2024,07-02-2024,08-02-2024,09-02-2024,10-02-2024,11-02-2024,12-02-2024,13-02-2024,14-02-2024,15-02-2024,16-02-2024,17-02-2024,18-02-2024,19-02-2024,20-02-2024,21-02-2024,22-02-2024,23-02-2024,24-02-2024,25-02-2024,26-02-2024,27-02-2024,28-02-2024,29-02-2024,01-03-2024,02-03-2024,03-03-2024,04-03-2024,05-03-2024,06-03-2024,07-03-2024,08-03-2024,09-03-2024,10-03-2024,11-03-2024,12-03-2024,13-03-2024,14-03-2024,15-03-2024,16-03-2024,17-03-2024,18-03-2024,19-03-2024,20-03-2024,21-03-2024,22-03-2024,23-03-2024,24-03-2024,25-03-2024,26-03-2024,27-03-2024,28-03-2024,29-03-2024,30-03-2024,31-03-2024,01-04-2024,02-04-2024,03-04-2024
225,Нижегородская,1698.0,МЦК,11946.0,17623.0,7216.0,11595.0,10309.0,8088.0,3980.0,19398.0,9136.0,11620.0,2284.0,6661.0,19888.0,19873.0,1129.0,1607.0,8949.0,6219.0,13573.0,1401.0,13486.0,15624.0,16524.0,12704.0,2133.0,15556.0,8235.0,6624.0,6135.0,3994.0,6969.0,4074.0,2194.0,3622.0,9578.0,9819.0,16467.0,13419.0,12732.0,16839.0,7498.0,17704.0,7534.0,17171.0,4686.0,9881.0,7128.0,12870.0,12076.0,5661.0,13759.0,15266.0,10014.0,17686.0,5329.0,17107.0,17314.0,13439.0,4526.0,8272.0,15321.0,2120.0,9966.0,10545.0,16999.0,10686.0,2421.0,7244.0,7087.0,15208.0,2694.0,11366.0,13491.0,2071.0,6125.0,9084.0,18698.0,8635.0,13698.0,16806.0,5429.0,8117.0,13299.0,9111.0,11954.0,11515.0,9490.0,6891.0,6873.0,1463.0,14374.0,18432.0,1668.0,12855.0
280,Нижегородская,1779.0,НЕКРАСОВСКАЯ,7458.0,6559.0,19477.0,12002.0,4202.0,18498.0,7095.0,18654.0,15480.0,11767.0,17188.0,17947.0,17074.0,11645.0,5999.0,18855.0,15946.0,4579.0,17689.0,3759.0,4525.0,11311.0,7078.0,3338.0,7837.0,9882.0,4303.0,8596.0,15051.0,2706.0,16437.0,17276.0,2868.0,13907.0,18187.0,2921.0,11740.0,4023.0,6154.0,4555.0,11199.0,6159.0,18947.0,16334.0,18026.0,2660.0,1079.0,14213.0,12079.0,9672.0,12605.0,10318.0,9305.0,19275.0,11162.0,9568.0,2417.0,3940.0,2144.0,16201.0,12128.0,4429.0,7514.0,12105.0,16502.0,12366.0,7011.0,3397.0,1135.0,1066.0,19712.0,17315.0,6365.0,15900.0,9783.0,8578.0,1397.0,4420.0,7879.0,10707.0,19272.0,1151.0,8269.0,16613.0,5031.0,9149.0,18366.0,4326.0,5590.0,17308.0,9557.0,19813.0,4503.0,7107.0
312,Нижегородская,1749.0,БКЛ,18969.0,10092.0,9394.0,6940.0,15306.0,12815.0,15196.0,14962.0,2221.0,13599.0,1590.0,8924.0,7955.0,15242.0,19303.0,1869.0,7672.0,11713.0,8182.0,19794.0,1442.0,12576.0,6843.0,14566.0,15631.0,7870.0,7569.0,1133.0,3895.0,8339.0,2666.0,19598.0,4240.0,2496.0,15955.0,7376.0,15209.0,18637.0,18263.0,7872.0,1810.0,17746.0,5827.0,17276.0,5695.0,6614.0,13179.0,10875.0,11773.0,11293.0,17672.0,5054.0,18325.0,18140.0,7204.0,5722.0,2457.0,11862.0,16564.0,3901.0,1791.0,16733.0,16471.0,2262.0,4714.0,10350.0,3554.0,7211.0,17602.0,17563.0,9895.0,14829.0,17644.0,19309.0,12890.0,19078.0,15011.0,2549.0,2542.0,14926.0,19396.0,2534.0,5186.0,12381.0,13410.0,17643.0,11618.0,18369.0,13183.0,1577.0,2667.0,6879.0,5763.0,18534.0


In [9]:
data.iloc[:, :3]

,Станция,Номер линии,Линия
0,Бульвар Рокоссовского,8.0,СОКОЛЬНИЧЕСКАЯ
1,Черкизовская,8.0,СОКОЛЬНИЧЕСКАЯ
2,Преображенская площадь,8.0,СОКОЛЬНИЧЕСКАЯ
3,Сокольники,8.0,СОКОЛЬНИЧЕСКАЯ
4,Красносельская,8.0,СОКОЛЬНИЧЕСКАЯ
5,Комсомольская,8.0,СОКОЛЬНИЧЕСКАЯ
6,Красные ворота,8.0,СОКОЛЬНИЧЕСКАЯ
7,Чистые пруды,8.0,СОКОЛЬНИЧЕСКАЯ
8,Лубянка,8.0,СОКОЛЬНИЧЕСКАЯ
9,Охотный ряд,8.0,СОКОЛЬНИЧЕСКАЯ
